In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini")
response = llm.invoke([HumanMessage(content="잘 지냈어?")])
print(response.content)

네, 감사합니다! 당신은 잘 지내셨나요? 어떤 이야기를 나누고 싶으신가요?


In [3]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool
def get_current_time(timezone:str, location: str) -> str:
    """현재 시각을 반환하는 함수
    
    Args:
        timezone (str): 타임존(예: 'Asia/Seoul'). 실제 존재해야 함
        location (str): 지역명, 타임존은 모든 지명에 대응되지 않으므로 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재 시각 {now}'
    print(location_and_local_time)
    return location_and_local_time

In [4]:
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

llm_with_tools = llm.bind_tools(tools)

In [5]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇 시야?"),
]

response = llm_with_tools.invoke(messages)
messages.append(response);

print(messages);

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_twTGuz0buGLB4bZY5Biw2qQm', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 130, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CNAXapUbqSYhf5WF7gtDUcN3MuInC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--71736291-810d-4297-85de-440f3f9a690a-0', tool_call

In [6]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재 시각 2025-10-05 13:21:42


In [7]:
from pydantic import BaseModel, Field

class StockhistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: Id, 1mo, 1y)")
    

In [8]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockhistoryInput) -> str:
    """주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(stock_history_input.period)
    history_md = history.to_markdown()
    
    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)


In [9]:
messages.append(HumanMessage("테슬라는 한 달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

content='' additional_kwargs={'tool_calls': [{'id': 'call_QVJ7jWTeahTgEl4sueG0Itpl', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 275, 'total_tokens': 302, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CNAXkkPI1fZd4U8CQoYRjvQvbHcav', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--33f43dc3-4d59-46c9-ad67-e740bc1270c2-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_QVJ7jWTeahTgEl4sueG0Itpl', 'type': 'tool_call'}] usage_metadata={'inp

In [10]:
for c in llm.stream([HumanMessage("잘 지냈어? 한국 사회의 문제점이 무엇인지 이야기해줘.")]):
    print(c.content, end='|')

|저|는| 잘| 지|내|고| 있습니다|.| 한국| 사회|의| 문제|점|에| 대해| 몇| 가지|를| 이야기|해|보|겠습니다|.

|1|.| **|고|령|화| 사회|**|:| 한국|은| 빠|른| 속|도로| 고|령|화|가| 진행|되고| 있습니다|.| 이는| 사회|保障| 시스템|에| 부담|을| 주|고|,| 젊|은| 세|대|의| 경제|적| 부담|이| 커|지는| 문제|를| 초|래|할| 수| 있습니다|.

|2|.| **|청|년| 실|업|**|:| 고|학|력| 청|년|층|의| 실|업|률|이| 높|아|지고| 있으며|,| 많은| 젊|은|이|들이| 안정|적인| 일|자리| 찾|기에| 어려|움을| 겪|고| 있습니다|.| 이는| 사회| 불|안|정|과| 불|만|을| 야|기|할| 수| 있습니다|.

|3|.| **|주|거| 문제|**|:| 대|도|시| 특히| 서울|의| 주|택| 가격|이| 급|등|하면서| 젊|은| 세|대|가| 주|거| 안정|성을| 느|끼|기| 어려|워|지고| 있습니다|.| 이는| 가|계| 부담|을| 증가|시키|고| 삶|의| 질|에| 큰| 영향을| 미|칠| 수| 있습니다|.

|4|.| **|사회|적| 격|차|**|:| 경제|적| 불|평|등|이| 심|화|되고| 있으며|,| 이|로| 인해| 사회|적| 갈|등|이| 심|화|되고| 있습니다|.| 소|득|의| 불|균|형|은| 사회|적| 통|합|을| 저|해|하는| 요|인이| 될| 수| 있습니다|.

|5|.| **|정|신| 건강| 문제|**|:| 높은| 경쟁|과| 스트|레스|의| 사회|적| 분위|기| 속|에서| 정신| 건강| 문제가| 심|각|해|지고| 있습니다|.| 특히| 젊|은| 세|대|의| 우|울|증| 및| 불|안| 장애|가| 증가|하고| 있습니다|.

|6|.| **|성|차|별|과| 성|폭|력| 문제|**|:| 한국| 사회|에서는| 여|전히| 성|차|별|과| 성|폭|력| 문제가| 심|각|하게| 존재|하고| 있으며|,| 이에| 대한| 인|식| 개선|과| 법| 제|도가| 필요|합니다|.

|이| 외|에도| 여러| 가지| 문제|들이

In [12]:
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇 시야?"),
]

response = llm_with_tools.stream(messages)

is_first = True
for chunk in response:
    print("chunk type: ", type(chunk))
    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk
        
    print("content: ", gathered.content, "tool_call_chunk", gathered.tool_calls)
    
messages.append(gathered)

chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_6PisPBfdRvZGNo0l7kodsaHW', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_6PisPBfdRvZGNo0l7kodsaHW', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_6PisPBfdRvZGNo0l7kodsaHW', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_6PisPBfdRvZGNo0l7kodsaHW', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_6PisPBfdRvZGNo0l7kodsaHW', 'type': 'tool_c

In [13]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재 시각 2025-10-05 13:38:37


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_6PisPBfdRvZGNo0l7kodsaHW', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--e8d9fee6-3b38-4d59-86e9-cf7fd473b372', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_6PisPBfdRvZGNo0l7kodsaHW', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_6PisPBfdRvZGNo0l7kodsaHW', 'index': 0, 'type': 'tool_call_chunk'}]),
 ToolMessage(content='Asia/

In [14]:
for c in llm_with_tools.stream(messages):
    print(c.content, end="|")

|부|산|은| 지금| |202|5|년| |10|월| |5|일| |13|시| |38|분| |37|초|입니다|.||